In [ ]:
%matplotlib inline
import copy
import qutip
from qutip import Qobj
import numpy as np
import math, cmath
import matplotlib.pyplot as plt
from tqdm import tqdm, trange

from src.stateobj import Physics
import src.utilities as use

import os

from IPython.display import Markdown, display

In [ ]:
TIMESTEPS = 300
TIMEDELTA = 1.0
OMEGA = 1.0  # Strength of Interaction

D = 20

p = Physics(dimension=D, interaction_strength=OMEGA, interaction_time=TIMEDELTA)

th = OMEGA * TIMEDELTA
alpha = complex(1 / math.sqrt(1 + 2*np.e), 0)
beta = cmath.sqrt(1 - alpha**2)
phi = np.pi/2
# Phase shifts
delta1 = 0
delta2 = -phi

In [ ]:
eta = use.create_ancilla_qobj(alpha, beta, phi)
rho1 = use.create_system_qobj('thermal', n=1, n_dims=D)
rho2 = use.create_system_qobj('thermal', n=1, n_dims=D)
rho = qutip.tensor(rho1, rho2)

In [ ]:
ga = 2 * alpha ** 2
gb = beta**2 * (1 + np.cos(phi))

In [ ]:
def commutator(A: Qobj | np.ndarray, B: Qobj | np.ndarray, kind='regular'):
    if kind == 'regular':
        return A*B - B*A
    elif kind == 'anti':
        return A*B + B*A

def dissipator(X: Qobj | np.ndarray, system: Qobj | np.ndarray, kind='regular'):
    if kind == 'regular':
        sandwich = X * system * X.dag()
    elif kind == 'anti':
        sandwich = X.dag() * system * X
    comm = qutip.commutator(X.dag()*X, system, kind='anti')
    return sandwich - 1/2 * comm

def master_equation(system, ga, gb):
    # Bosonic Operators
    C = p.C
    Cp = p.Cp
    S = p.S
    Sd = p.S.dag()
    first_line = 0.5*dissipator(qutip.tensor(C, C) - 2*qutip.tensor(S, Sd), system)
    first_line += dissipator(qutip.tensor(C, S) + qutip.tensor(S, Cp), system)
    second_line = 0.5*dissipator(qutip.tensor(Cp, Cp) - 2*qutip.tensor(Sd, S), system)
    second_line += dissipator(qutip.tensor(Cp, Sd) + qutip.tensor(Sd, C), system)
    return ga * first_line + gb * second_line

def evolve(system, ga, gb):
    delta_s = master_equation(system, ga, gb)
    return system + delta_s

def unitary_evolve(system):
    sigma = qutip.tensor(system, eta)
    exponent = -1j * TIMEDELTA * ( p.V1 + p.V2 )
    U = exponent.expm()
    sigma_evolution = U * sigma * U.dag()
    return sigma_evolution.ptrace([0, 1])

def hilbert_is_good(system, check):
    """Check if the Hilbert space truncation is still valid"""
    threshold = 9e-4
    if check == 'unitary':
        trace = system.tr()
        return abs(trace - 1) < threshold
    elif check == 'small_tail':
        last_diagonal_element = system.diag()[-1]
        return last_diagonal_element < threshold

## One-Step Evolution

In [ ]:
system = copy.deepcopy(rho)
display(system)
# Master Equation Evolution
delta_system = master_equation(system, ga, gb)
print('Master Equation Evolution')
display(system + delta_system)

In [ ]:
# Unitary Evolution
sigma = qutip.tensor(system, eta)
exponent = 1j * TIMEDELTA * ( p.V1 + p.V2 )
U = exponent.expm()
sigma_evolved = U * sigma * U.dag()
print('Unitary Evolution')
display(sigma_evolved.ptrace([0, 1]))

In [ ]:
entropies = []
rho_unitary = copy.deepcopy(rho)
entropies_unitary = []
purities = []

In [ ]:
# Search for file
files = os.listdir('../objects/')
t = 0
try:
    # Extract the time from the file name
    time = max([int(f.split('_')[-2][1:]) for f in files if f.startswith('rho_20230707')])
    # Load partial data
    name = f'20230628_t{time}_d{D}'
    rho = qutip.fileio.qload('objects/rho_' + name + '.qu')
    entropies = np.load('objects/entropies_' + name + '.npy').tolist()
    purities = np.load('objects/purities_' + name + '.npy').tolist()
    print(f'Loaded values fot t={time}')
except FileNotFoundError as e:
    # Files not found, run evolution
    print(e)
    for t in trange(TIMESTEPS):
        rho = evolve(rho, ga, gb)
        entropies.append(qutip.entropy_vn(rho))
        purities.append(rho.purity())
    time = t + 1
except ValueError as e:
    # Files not found, run evolution
    print(e)
    for t in trange(TIMESTEPS):
        rho = evolve(rho, ga, gb)
        rho_unitary = unitary_evolve(rho_unitary)
        entropies.append(qutip.entropy_vn(rho))
        entropies_unitary.append(qutip.entropy_vn(rho_unitary))
        purities.append(rho.purity())
    time = t + 1

In [ ]:
# Plot quantities
plt.plot(entropies, label='Entropy')
plt.plot(entropies_unitary, label='Entropy Unitary')
plt.legend()
plt.show()

In [ ]:
# Evolve again
for t in trange(10 * TIMESTEPS):
    rho = evolve(rho, ga, gb)
    entropies.append(qutip.entropy_vn(rho))
    purities.append(rho.purity())
time += t + 1

In [ ]:
time

In [ ]:
# Save partial data
name = f'20230707_t{time}_d{D}'
qutip.fileio.qsave(rho, 'objects/rho_' + name + '.qu')
np.save('objects/entropies_' + name + '.npy', np.array(entropies))
np.save('objects/purities_' + name + '.npy', np.array(purities))

In [ ]:
rho